In [4]:
import os
import requests
import re
from googlesearch import search 
from tabula import read_pdf
import urllib.parse

dirName = "announcements"
dirPath = ".\\" + dirName + "\\"

try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  announcements  already exists


In [72]:
def download_file(url):
    file = requests.get(url, stream=True)
    headers = file.headers['content-disposition']
    filename = re.findall("filename=\"(.+)\"", headers)[0]
    location = os.path.relpath(dirPath + filename)
    open(location, 'wb').write(file.content)
    del file

In [70]:
query = "\"S&P 500 INDEX –\" \"Set to Join\" site:spindices.com filetype:pdf"
results = search(query, tld='com', lang='en', num=10, start=0, stop=None, pause=2.0)

In [74]:
for result in results:
    download_file(result)
    print(result)

https://us.spindices.com/documents/indexnews/announcements/20200122-1077359/1077359_5payc.pdf?force_download=true
https://us.spindices.com/documents/indexnews/announcements/20190709-957359/957359_hat5mobile.pdf?force_download=true
https://us.spindices.com/documents/indexnews/announcements/20200227-1098962/1098962_Final5Ingersol4Cimar.pdf?force_download=true
https://www.spindices.com/documents/indexnews/announcements/20190314-892576/892576_21foxspindis100adbe.pdf
https://www.spindices.com/documents/indexnews/announcements/20191127-1041559/1041559_BerkleyRJoe546.pdf
https://www.spindices.com/documents/indexnews/announcements/20190801-969367/969367_1leidos546foot6fizz6.pdf
https://us.spindices.com/documents/indexnews/announcements/20190624-948566/948566_mktxaaxnmlab546usna6xncr6.pdf?force_download=true
https://www.spindices.com/documents/indexnews/announcements/20190326-898958/898958_1dowspin100trex400.pdf
https://www.spindices.com/documents/indexnews/announcements/20190926-1004359/100435

ConnectionError: HTTPConnectionPool(host='stg.spindices.com', port=80): Max retries exceeded with url: /documents/indexnews/announcements/20150706-204357/204357_dollarjackcontact546.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000021D19A414C8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [160]:
import pandas as pd
columns = ['type', 'name', 'ticker', 'announcement_date', 'implementation_date']
df = pd.DataFrame(columns=columns)

In [151]:
# import camelot
# import ghostscript
# from tika import parser
# import textract
# from tika import unpack
import pdfplumber

for filename in os.listdir(dirPath):
#     print(filename)
#     file = open(dirPath + filename, 'rb')
#     data = file.read()
#     print(data)
#     text = textract.process(dirPath + filename, method='tesseract', language='eng')
#     print(text)
#     raw = parser.from_file(dirPath + filename)
#     print(raw['metadata'])
#     data = unpack.from_file(dirPath + filename)
#     print(data)
    pdf = pdfplumber.open(dirPath + filename)
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            if any("S&P 500 INDEX" in str(item) for item in table[0]):
                print(table)
                break
                

    #     announcement_date = re.findall("New York, NY, () –", raw['content'])[0]
#     print(announcement_date)
#     break
#     tables = camelot.read_pdf(dirPath + filename, pages = "1-end")
#     for table in tables:
#         print(table.df)
#     dfs = read_pdf(dirPath + filename, pages = "all", multiple_tables = True, guess=False)
#     for df in dfs:
# #         if "S&P 500 INDEX" in df.values:
# #         if "S&P 500 INDEX" in df[1][0]:
#         print(df)
#     break

[['S&P 500 INDEX – September 26, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'NVR', 'Consumer Discretionary', 'Homebuilding'], ['DELETED', 'Jefferies \nFinancial Group', 'Financials', 'Multi-Sector Holdings']]
[['S&P 500 INDEX – October 3, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Las Vegas \nSands', 'Consumer Discretionary', 'Casinos & Gaming'], ['DELETED', 'Nektar \nTherapeutics', 'Health Care', 'Pharmaceuticals']]
[['S&P 500 INDEX – December 5, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'W.R. Berkley', 'Financials', 'Property & Casualty \nInsurance'], ['DELETED', 'Viacom', 'Communication Services', 'Movies & Entertainment']]
[['S&P 500 INDEX – December 9, 2019', None, None, None], ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'], ['ADDED', 'Old Dominion \nFreight Line', 'Industrials', 'Trucking'], ['DELETED'

KeyboardInterrupt: 

In [152]:
table

[['S&P 500 INDEX – November 17, 2015', None, None, None],
 ['', 'COMPANY', 'GICS ECONOMIC SECTOR', 'GICS SUB-INDUSTRY'],
 ['ADDED', 'Synchrony Financial', 'Financials', 'Consumer Finance'],
 ['DELETED', 'Genworth Financial', 'Financials', 'Multi-line Insurance']]

In [153]:
table[0]

['S&P 500 INDEX – November 17, 2015', None, None, None]

In [175]:
def get_ticker_by_name(stock_name):
    '''
    Takes in a stock_name in string,
    Returns the ticker in string
    '''
    encoded_stock_name = urllib.parse.quote(stock_name)
    url = "https://s.yimg.com/aq/autoc?query=" + encoded_stock_name + "&lang=en-US"
    response = requests.get(url)
    json_response = response.json()
    results = json_response['ResultSet']['Result']
    if len(results) == 0:
        print('No ticker found.')
        return None
    else:
        return results[0]['symbol']

In [176]:
table[2]
for i in range(2, len(table)):
    print(table[i])
    change_type = table[i][0]
    name = table[i][1].replace('\n', '')
    ticker = get_ticker_by_name(name)
    row = [{'type': change_type, 'name': name, 'ticker': ticker}]
    df = df.append(pd.DataFrame(row), ignore_index=True)

['ADDED', 'Synchrony Financial', 'Financials', 'Consumer Finance']
['DELETED', 'Genworth Financial', 'Financials', 'Multi-line Insurance']


In [173]:
name = 'Nektar \nTherapeutics'.replace('\n', '')
name

'Nektar Therapeutics'

In [177]:
df

,type,name,ticker,announcement_date,implementation_date
0,ADDED,Synchrony Financial,NaN,NaN,NaN
1,DELETED,Genworth Financial,NaN,NaN,NaN
2,ADDED,Synchrony Financial,NaN,NaN,NaN
3,DELETED,Genworth Financial,NaN,NaN,NaN
4,ADDED,Synchrony Financial,SYF,NaN,NaN
5,DELETED,Genworth Financial,GNW,NaN,NaN


In [144]:
# from pandas_datareader import data as pdr

# import yfinance as yf
# yf.pdr_override() # <== that's all it takes :-)

# # download dataframe
# data = pdr.get_data_yahoo("Arista Networks", start="2017-01-01", end="2017-04-30")
# stock_name = "Arista Networks"
stock_name = "random shiz"
encoded_stock_name = urllib.parse.quote(stock_name)
# url = "https://s.yimg.com/aq/autoc?query=" + encoded_stock_name + "&region=CA&lang=en-CA&callback=YAHOO.util.ScriptNodeDataSource.callbacks"
url = "https://s.yimg.com/aq/autoc?query=" + encoded_stock_name + "&lang=en-US"
response = requests.get(url)

In [145]:
json_response = response.json()

In [142]:
json_response['ResultSet']['Result'][0]['symbol']

'ANET'

In [148]:
results = json_response['ResultSet']['Result']

In [149]:
if len(results) == 0:
    print('No ticker found.')
else:
    ticker = results[0]['symbol']

No ticker found.


In [146]:
json_response

{'ResultSet': {'Query': 'random shiz', 'Result': []}}